In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/demand-forecasting-kernels-only/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sales_train = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/train.csv', index_col = 0)
sales_test = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv',index_col = 1)
sales_train.index = pd.to_datetime(sales_train.index)
sales_test.index = pd.to_datetime(sales_test.index)
display(sales_train.sample(10))
display(sales_test.sample(10))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py

from dateutil.relativedelta import relativedelta
from scipy.optimize import minimize

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as sts
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error, mean_squared_log_error

from itertools import product
from tqdm import tqdm_notebook
import itertools

import warnings

## Visualizing Sales & their Properties

In [ ]:
# To infer the typical number of items sold each day
plt.figure(figsize = (15,7))
plt.hist(sales_train['sales'], bins = 10)
plt.title('Typical number of items sold each day')
plt.show()

In [ ]:
store_item_df = sales_train.copy()
# First, let us filterout the required data
store_id = 10   # Some store
item_id = 40    # Some item
print('Before filter:', store_item_df.shape)
store_item_df = store_item_df[store_item_df.store == store_id]
store_item_df = store_item_df[store_item_df.item == item_id]
print('After filter:', store_item_df.shape)
#display(store_item_df.head())

plt.figure(figsize = (15,7))
plt.plot(store_item_df.index, store_item_df.sales)
plt.grid(True)
plt.title('Daily Item sales at one store')
plt.show()

In [ ]:
# The daily sales values seem very sporadic, we are now gonna plot the sum value of sales over a week
stores_sales_df1 = sales_train.copy()
stores = pd.DataFrame(stores_sales_df1.groupby(['date', 'item']).sum()['sales']).unstack()
stores = stores.resample('W',label='left').sum()
stores.sort_index(inplace = True)

display(stores.sample(10))

stores.plot(figsize=(15,7), title='Weekly Item Sales', legend=None)
plt.show()

In [ ]:
# The daily sales values seem very sporadic, we are now gonna plot the sum value of sales over a week
stores_sales_df = sales_train.copy()
stores = pd.DataFrame(stores_sales_df.groupby(['date', 'store']).sum()['sales']).unstack()
stores = stores.resample('W',label='left').sum()
stores.sort_index(inplace = True)

display(stores.sample(10))
display(stores.head(10))

stores.plot(figsize=(15,7), title='Weekly Store Sales', legend=None)
plt.show()

In [ ]:
# Need to visualise the trends, seasonality and other features (on both additive and multiplicative scales) here.

date_sales = sales_train.drop(['store','item'], axis=1).copy() 
y = date_sales['sales'].resample('MS').mean() 
y['2017':] #sneak peak
y.plot(figsize=(15, 7))

decomposition = sm.tsa.seasonal_decompose(y, model='additive')
decomposition.plot()

# Still have to make sense of the plots and describe it, also, need to try and plot these components on a weekly basis, but on a smaller time scale as well (Say 3 months).

In [ ]:
#We can also visualize our data using a method called time-series decomposition that allows us to decompose our time series into three distinct components: 
#trend, seasonality, and noise.
# Here, we are using multiplicative models of decomposition, instead of additive ones
decomposition = sm.tsa.seasonal_decompose(y, model='multiplicative')
decomposition.plot()

### Supporting Functions
These functions will be used to assist creation of data, to implement loss metrics that aren't implicit in scipy library, and a module to plot the results

In [ ]:
def create_data(df, store_id, item_id):
    '''
    This function creates a series containing the sales values of a particular item of a particular store (as prescribed in the argument values). This will return a pandas series
    
    '''
    new_series = df.copy()
    new_series = df[df.store == store_id]
    new_series = df[df.item == item_id]
    return new_series

def mean_absolute_percentage_error(y_true, y_pred, multioutput = 'raw_values'):
    '''
    This function returns the mean absolute percentage error of the values in form of a numpy array, if the multoutput is set to 'raw_values', and returns a single float value of 
    the average of loss, if the multioutput is set to 'uniform_average'
    
    Args:
        y_true (iterable) -> Representing the actual values of the output
        y_pred (iterable) -> Representing the predicted values of the output
        multioutput (string) -> Could either be 'raw_values', or 'uniform_average'
        
    Returns:
        A numpy array or a single float value depending on the multioutput argument.
    '''
    if multioutput == 'raw_values':
        return np.divide(np.abs(y_true - y_pred), np.abs(y_true)) * 100
    if multioutput == 'uniform_average':
        return np.mean(np.divide(np.abs(y_true - y_pred), np.abs(y_true)) * 100)

def total_error(y_true, y_pred, metric):
    '''
    This function returns the total error of the predicted values when evaluated against the actual values. It returns a single float variable representing that error
    
    '''
    if metric<0 or metric>4:
        raise ValueError('The loss metric should be between 1 and 4, included')
    if metric == 0:
        error_term = mean_absolute_error(actual_value, pred_value, multioutput = 'uniform_average')
    if metric == 1:
        error_term = mean_squared_error(actual_value, pred_value, multioutput = 'uniform_average')
    if metric == 2: 
        error_term = mean_absolute_percentage_error(actual_value, pred_value, multioutput = 'uniform_average')
    if metric == 3:
        error_term = r2(actual_value, pred_value, multioutput = 'uniform_average')
    if metric == 4:
        error_term = smape(actual_value, pred_value, multioutput = 'uniform_average')
    return error_term
    



In [ ]:
def smape(y_true, y_pred, multioutput = 'raw_values'):
    '''
    This function returns the symmetric mean absolute percentage error of the values in form of a numpy array, if the multoutput is set to 'raw_values', and returns a single float value of 
    the average of loss, if the multioutput is set to 'uniform_average'
    The formula for SMAPE has been defined here - https://www.forecastpro.com/Trends/forecasting101August2011.html
    
    Args:
        y_true (iterable) -> Representing the actual values of the output
        y_pred (iterable) -> Representing the predicted values of the output
        multioutput (string) -> Could either be 'raw_values', or 'uniform_average'
        
    Returns:
        A numpy array or a single float value depending on the multioutput argument.
        
    '''
    if multioutput == 'raw_values':
        return np.divide(np.abs(y_true - y_pred), np.abs((y_true + y_pred)/2) * 100)
    else:
        return np.mean(np.divide(np.abs(y_true - y_pred), np.abs((y_true + y_pred)/2)) * 100)
    

In [ ]:
# A special point of note should be made here, if we have the data, for the actual points (even though we are making predictions for those time steps, and not using the actual data)
# we can very conveniently use this very function to make a reasonable plot. We will not have to worry about the broadcasting problem in arrays of different sizes, because, we will
# simply add the know actual values to the actual_value array for the prediction period as well. 
# Also, we need to define seperate plotting functions for each of the models becuase the predictions they make won't necessarily have actual values corresponding to them 
# and we will have to use particular plot methods for each of those methods (for eg, triple exponential smoothing can be predicted using brutlag method)

def plot_results(pred_value, actual_value, plot_intervals = False, scale = 2, plot_anomalies = False, metric = 4, pred_start = -1):
    '''
    This function plots the results of a time series prediction. It can also plot the intervals and anomalies, if so directed.
    
    Args:
        pred_value (iterable) -> Integer values of the predicted values of series
        actual_value (iterable) -> Integer values of the actual values of the series
        plot_intervals (boolean) -> Decides if the lower and upper bounds are to be plotted.
        plot_anomalies (boolean) -> Decides if the anomalies are to be plotted.
        scale (float) -> An hyperparameter for interval deduction.
        metric (integer) -> To decide the loss metric to be used
            0 - MAE
            1 - MSE
            2 - MAPE
            3 - R2
            4 - SMAPE
        pred_start (integer) -> A negative integer referring to the number of time steps that have been used as test data. 
        
    '''
    
    plt.figure(figsize = (15,7))
    plt.title('Prediction values for sales')
    plt.plot(pred_value, "g", label = "Predicted values")
    
    # Plot the confidence intervals of the smoothed values
    if plot_intervals:
        if metric<0 or metric>4:
            raise ValueError('The loss metric should be between 1 and 4, included')
        if metric == 0:
            error_term = mean_absolute_error(actual_value, pred_value, multioutput = 'raw_values')
        if metric == 1:
            error_term = mean_squared_error(actual_value, pred_value, multioutput = 'raw_values')
        if metric == 2: 
            error_term = mean_absolute_percentage_error(actual_value, pred_value, multioutput = 'raw_values')
        if metric == 3:
            error_term = r2(actual_value, pred_value, multioutput = 'raw_values')
        if metric == 4:
            error_term = smape(actual_value, pred_value, multioutput = 'raw_values')
        deviation = np.std(actual_value - pred_value)
        lower_bound = pred_value - (error_term + scale*deviation)
        upper_bound = pred_value + (error_term + scale*deviation)
        plt.plot(upper_bound, "r--", label = "Upper Bound / Lower Bound")
        plt.plot(lower_bound, "r--")
        
        if plot_anomalies:
            anomalies = pd.DataFrame(index = actual_value.index, columns = actual_value.columns)
            anomalies[actual_value<lower_bound] = actual_value[actual_value<lower_bound]
            anomalies[actual_value>upper_bound] = actual_value[actual_value>upper_bound]
            plt.plot(anomalies, "ro", makersize = 1)
        
    plt.plot(actual_value, label = "Actual Values")
    plt.axvspan(len(actual_value) + pred_start, len(actual_value), alpha = 0.4, color = 'light grey')
    plt.legend(loc = "upper left")
    # See if the total error term can be added to the plot (to compare the results of different models and different evaluation metrics)
    plt.grid(True)
    plt.show()
        
        

## Exponential Smoothing
The problem with exponential smoothing is that we can't use it to predict the trends of the data for very long periods (6 months in our case), and this limits the utility of this model, nonetheless, this can be used to make a naive prediction on a short term basis and serves as as a good estimate of a conservative prediction which can then be used to compare the volatility and credibility of the predictions made by our long term predictions. Think of it this way, if we were to make predictions for, lets say stock prices, and there is and a certain incident that causes the time series to behave unusually (like a chain of events set in motion due to some scandal or something), then we would want our model to quickly adopt itself to the new environment it has found itself in. Of course, there won't be enough data of such cataclysmic changes to make meaningful long term predictions, and in those scenarios, a manually tuned expectancy value derived from Exponential Smoothing will prove itself to be extremely useful.

#### Smoothing parameters
We will try and find the values of the smoothing parameters by passing the input values through a truncated conjugate gradient optmisation algorithm. This will help us ensure that we aren't just tuning those hyperparameters by ourselves, instead they are being set on a formal basis of how beneficial they are to the learning process.

The code for triple exponential smoothing will be completely different. To provide long term predictions along with deviations that are tuned on the basis of how far long the prediction horizon is. Also, the confidence intervals are derived using the brutlag method, and the parameter values are derived using the optimization function, rather than feeding them manually. I will create a seperate class for these predictions.

In [ ]:
class Single_ES:
    
    """
    This model object will produce the Single Exponential smoothing values. Since it does not have the abilities to either smooth or to add trend, it can only be used to make a single 
    meaningful prediction. That is, just one time step can be predicted.
    Args:
        series (iterable) -> Actual time series
        alpha (float) -> Represents the parameter for single exponential smoothing
        scaling_factor (float) -> sets the width of the confidence interval
        
    Returns:
        result (iterable) -> A numpy array containing the results of smoothing
    
    """
    
    
    def __init__(self, series, alpha = 0.9, scaling_factor=2):
        self.series = series
        self.alpha = alpha
        # Scaling factor will only be used, if the plot is done here in this class
        self.scaling_factor = scaling_factor
        
          
    def single_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.result = [series[0]] # first value is same as series
        for t in range(1, len(series)):
            self.result.append(self.alpha * self.series[t] + (1 - self.alpha) * self.result[t-1])
        # Since we can't make more than one prediction, we can either print the plot here, or we can print the plot from the calling function where the 'model' object is created. 
        # There's no difference in the plots drawn from either.
        return np.array(self.result)

In [ ]:
class Double_ES:
    
    """
    This class performs double exponential smoothing and updates the deviation, upper bound, and lower bound values for each time step. It also plots the results with its own function.
    There is no limit to the prediction horizon, however, since we are only relying on the trend update to make the predictions, it wouldn't be wise to set the n_preds value to be too 
    large.
    
    Args:
        series (iterable) -> Actual values array
        alpha, beta (float) -> Double exponential smoothing parameters
        n_preds (int) -> predictions horizon
        scaling_factor (float) - sets the width of the confidence interval and is used to determine the upper bound and lower bound
    
    """
    
    
    def __init__(self, series, alpha, beta, n_preds, scaling_factor=2):
        self.series = series
        self.alpha = alpha
        self.beta = beta
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor
        
        
    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+1] - self.series[i])
        return sum /(len(self.series))

          
    def double_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBound = []
        self.LowerBound = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                
                self.PredictedDeviation.append(0)
                
                # Can't decide if I should add the error term (as I did in plot_results() function), into the evaluation of upper and lower bounds. I will have to rewrite that 
                # entire function here, and take metric as an argument as well. Will have to see the plot, to figure out if I should.
                self.UpperBound.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                self.LowerBound.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                continue
                
            if i >= len(self.series): # predicting
                m = i - len(self.series) + 1
                
                # We could apply smoothing to the 'm*trend' part of the code in the next line, it will ensure that the function isn't running just on final trend value, but is 
                # also considering the trend values of the earlier time steps. Will have to see the results to alter this part.
                self.result.append(smooth + m*trend)
                
                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                self.result.append(smooth+trend)
                
                # Instead of calculating the standard deviation of the results, we will here calculate the deviation using a variant of brutlag method, with the only difference being,
                # instead of using gamma as the parameter for deviation updata, we will use a fixed value of 0.5 (which we can adjust according to the results)
                self.PredictedDeviation.append(0.5 * np.abs(self.series[i] - self.result[i]) 
                                               + (1- 0.5)*self.PredictedDeviation[-1])
            
            # Again, the error term can be added here (to grant the model more versatility)
            self.UpperBound.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBound.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
        return np.array(self.result)
        
    def plot_Double_ES(self, plot_intervals = False, plot_anomalies = False):
        '''
        Function plots the results for DES. It can plot the results, lower bound, and upper bound even for time steps that do not have actual values given.
        This plot function is more geared towards the actual industry situations where we are deploying the model for predictions without having the 'actual observed values' given.
        
        args:
            plot_intervals (boolean) -> Defines if we want to plot the confidence intervals
            plot_anomalies (boolean) -> Defines if we want to plot the anomalies
        
        '''
        plt.figure(figsize = (15,7))
        plt.plot(self.results, "g", label = "Predicted Values")
        plt.label("Prediction values for sales using DES")
        
        if plot_intervals:
            plt.plot(self.UpperBound, "r--", alpha = 0.5, label = "Upper Bound/Lower Bound")
            plt.plot(self.LowerBound, "r--", alpha = 0.5)
            plt.fill_between(x = range(0,len(self.result)), y1 = self.UpperBound, y2 = self.LowerBound, alpha = 0.2, color = "grey")
            
        if plot_anomalies:
            self.anomalies = np.array([np.Nan]*len(self.series))
            self.anomalies[self.series.values<self.LowerBound[:len(self.series)]] = self.series.values[self.series.values<self.LowerBound[:len(self.series)]]
            self.anomalies[self.series.values>self.UpperBound[:len(self.series)]] = self.series.values[self.series.values>self.UpperBound[:len(self.series)]]
            plt.plot(self.anomalies, "o", markersize = 1, label = "Anomalies")
        
        plt.plot(self.series, label = "Actual Values")
        plt.legend(loc = 'upper left')
        plt.axvspan(len(self.series), len(self.result), alpha = 0.4, color = 'light grey')
        plt.grid(True)
        plt.axis('tight')
        plt.show()
        
        

In [ ]:
class Triple_ES:
    
    """
    In Triple Exponential Smoothing, we implement the Holt-Winters model, and derive the outliers using the brutlag method
    
    Args:
        series (iterbale) -> Actual values array
        slen (int) -> Length of a season
        alpha, beta, gamma (float) -> Holt-Winters model coefficients
        n_preds (int) -> predictions horizon
        scaling_factor (float) -> sets the width of the confidence interval by Brutlag (usually takes values from 2 to 3)
    
    """
    
    
    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor
        
        
    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   

          
    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBound = []
        self.LowerBound = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])
                
                self.PredictedDeviation.append(0)
                
                self.UpperBound.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                self.LowerBound.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                continue
                
            if i >= len(self.series): # predicting
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])
                
                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])
                
                # Deviation is calculated according to Brutlag algorithm.
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])
                     
            self.UpperBound.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBound.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i%self.slen])
        return np.array(self.results)
    
    
    def plot_Triple_ES(self, plot_intervals = False, plot_anomalies = False):
        '''
        Function plots the results for DES. It can plot the results, lower bound, and upper bound even for time steps that do not have actual values given.
        This plot function is more geared towards the actual industry situations where we are deploying the model for predictions without having the 'actual observed values' given.
        
        args:
            plot_intervals (boolean) -> Defines if we want to plot the confidence intervals
            plot_anomalies (boolean) -> Defines if we want to plot the anomalies
        
        '''
        plt.figure(figsize = (15,7))
        plt.plot(self.results, "g", label = "Predicted Values")
        plt.label("Prediction values for sales using Triple Exponential Smoothing")
        
        if plot_intervals:
            plt.plot(self.UpperBound, "r--", alpha = 0.5, label = "Upper Bound/Lower Bound")
            plt.plot(self.LowerBound, "r--", alpha = 0.5)
            plt.fill_between(x = range(0,len(self.result)), y1 = self.UpperBound, y2 = self.LowerBound, alpha = 0.2, color = "grey")
            
        if plot_anomalies:
            self.anomalies = np.array([np.Nan]*len(self.series))
            self.anomalies[self.series.values<self.LowerBound[:len(self.series)]] = self.series.values[self.series.values<self.LowerBound[:len(self.series)]]
            self.anomalies[self.series.values>self.UpperBound[:len(self.series)]] = self.series.values[self.series.values>self.UpperBound[:len(self.series)]]
            plt.plot(self.anomalies, "o", markersize = 1, label = "Anomalies")
        
        plt.plot(self.series, label = "Actual Values")
        plt.legend(loc = 'upper left')
        plt.axvspan(len(self.series), len(self.result), alpha = 0.4, color = 'light grey')
        plt.grid(True)
        plt.axis('tight')
        plt.show()
        


In [ ]:
# This cell is to include the data preparation part, where we do the train test split on a rolling basis. This function will be called by the proceeding optimization function and this 
# function will call the create_data function created above.



In [ ]:
# This cell will include the main wrapper part of the code, which will first call the optimization function to derive the parameter values, and then will create objects for each of the
# three types of ES models, and produce the results, then use those objects to call the plot function of TES and normally call the plot function for SES, DES. Here, I will also write 
# the code for identifying the anomalies by calling the necessary functions.

